In [17]:
import numpy as np
import pickle

from get_swing_foot_traj import *

In [18]:
class StepInfo():
    def __init__(self, index, state, time_vec, mode, footplant, swing=None):
        self.index = index
        self.state = state
        self.time_vec = time_vec
        self.mode = mode
        self.footplant = footplant
        self.swing = swing

with open('step_infos', 'rb') as step_infos_file:
    step_infos = pickle.load(step_infos_file)

In [19]:
class FootInfo():
    def __init__(self, time_vec, foot_left, foot_right):
        self.time_vec = time_vec
        self.foot_left = foot_left
        self.foot_right = foot_right
        
foot_infos = []

Move the COM above one foot

In [20]:
idx = 0
X_TORSO = 0.12

new_state = step_infos[idx].state
body_pos_rel = new_state[np.ix_([0, 2, 4]), :].reshape(3, -1)
body_pos_abs = body_pos_rel + step_infos[idx].footplant
stance_foot_pos_rel = np.array([[-1], [1], [-1], [1], [-1], [1]]) * new_state

l_foot = stance_foot_pos_rel
r_foot = l_foot + np.array([[2 * X_TORSO], [0], [0], [0], [0], [0]])
finfo = FootInfo(step_infos[idx].time_vec, l_foot, r_foot)
foot_infos.append(finfo)

Make a half step

In [21]:
idx = 1
SWING_HEIGHT = 0.1
SAMPLE_TIME = 0.005

new_state = step_infos[idx].state
body_pos_rel = new_state[np.ix_([0, 2, 4]), :].reshape(3, -1)
body_pos_abs = body_pos_rel + step_infos[idx].footplant
stance_foot_pos_rel = np.array([[-1], [1], [-1], [1], [-1], [1]]) * new_state

swing_abs = step_infos[idx].swing
foot_pos0 = swing_abs[:, 0] - body_pos_abs[:, 0]
foot_pos1 = swing_abs[:, -1] - body_pos_abs[:, -1]

time_stp0 = step_infos[idx].time_vec[0]
time_stp1 = step_infos[idx].time_vec[-1]
[q_swing_rel, qd_swing_rel, qdd_swing_rel] = get_swing_foot_traj(foot_pos0, foot_pos1,
                                                     SWING_HEIGHT, time_stp0, time_stp1, SAMPLE_TIME)
swing_pos = q_swing_rel + body_pos_abs
swing_foot_state = np.array([q_swing_rel[0, :], qd_swing_rel[0, :], q_swing_rel[1, :], qd_swing_rel[1, :],
                             q_swing_rel[2, :], qd_swing_rel[2, :]])

if step_infos[idx].mode == 'singlesupportleft':
    l_foot = stance_foot_pos_rel
    r_foot = swing_foot_state
    finfo = FootInfo(step_infos[idx].time_vec, l_foot, r_foot)
    foot_infos.append(finfo)
elif step_infos[idx].mode == 'singlesupportright':
    l_foot = swing_foot_state
    r_foot = stance_foot_pos_rel
    finfo = FootInfo(step_infos[idx].time_vec, l_foot, r_foot)
    foot_infos.append(finfo)

Consecutive Steps

In [22]:
for idx in range(2, len(step_infos)):
    
    curmode = step_infos[idx].mode
    new_state = step_infos[idx].state
    body_pos_rel = new_state[np.ix_([0, 2, 4]), :].reshape(3, -1)
    body_pos_abs = body_pos_rel + step_infos[idx].footplant
    stance_foot_pos_rel = np.array([[-1], [1], [-1], [1], [-1], [1]]) * new_state
    
    swing_abs = step_infos[idx].swing
    foot_pos0 = swing_abs[:, 0] - body_pos_abs[:, 0]
    foot_pos1 = swing_abs[:, -1] - body_pos_abs[:, -1]
    
    time_stp0 = step_infos[idx].time_vec[0]
    time_stp1 = step_infos[idx].time_vec[-1]

    [q_swing_rel, qd_swing_rel, qdd_swing_rel] = get_swing_foot_traj(foot_pos0, foot_pos1,
                                                         SWING_HEIGHT, time_stp0, time_stp1, SAMPLE_TIME)
    
    swing_pos = q_swing_rel + body_pos_abs
    swing_foot_state = np.array([q_swing_rel[0, :], qd_swing_rel[0, :], q_swing_rel[1, :], qd_swing_rel[1, :],
                                 q_swing_rel[2, :], qd_swing_rel[2, :]])
    
    if step_infos[idx].mode == 'singlesupportleft':
        l_foot = stance_foot_pos_rel
        r_foot = swing_foot_state
        finfo = FootInfo(step_infos[idx].time_vec, l_foot, r_foot)
        foot_infos.append(finfo)
    elif step_infos[idx].mode == 'singlesupportright':
        l_foot = swing_foot_state
        r_foot = stance_foot_pos_rel
        finfo = FootInfo(step_infos[idx].time_vec, l_foot, r_foot)
        foot_infos.append(finfo) 

In [23]:
with open('foot_infos', 'wb') as foot_infos_file:
    pickle.dump(foot_infos, foot_infos_file)

In [33]:
foot_infos[-1].foot_left.T

array([[-1.17770839e-01,  0.00000000e+00, -9.80722485e-02,
         0.00000000e+00, -6.80000000e-01,  0.00000000e+00],
       [-1.17770932e-01, -3.73267273e-05, -9.80639464e-02,
         3.31666882e-03, -6.79983276e-01,  6.67275832e-03],
       [-1.17771212e-01, -7.43730133e-05, -9.80391233e-02,
         6.60841902e-03, -6.79933440e-01,  1.32448717e-02],
       ...,
       [-1.19999542e-01, -8.17486176e-05,  9.99592732e-02,
         7.26377883e-03, -6.79918873e-01, -1.45471193e-02],
       [-1.19999858e-01, -4.47584198e-05,  9.99873855e-02,
         3.97701235e-03, -6.79975270e-01, -7.99517890e-03],
       [-1.19999989e-01, -7.48778075e-06,  9.99990018e-02,
         6.65327254e-04, -6.79998657e-01, -1.34259424e-03]])